In [ ]:
# set up environment
import os.path
import pathlib
import bmesh
import mathutils
import numpy as np
import math

# jupyter related imports
from IPython.display import Image

# blender related imports
import bpy

exchange_path = pathlib.Path().absolute()

In [ ]:
# create collection
collection = bpy.data.collections.new(name='JupyterCollection')

# to place in root container
bpy.context.scene.collection.children.link(collection)

In [ ]:
def clear_screen():
    # remove mesh Cube
    if "Sphere" in bpy.data.meshes:
        mesh = bpy.data.meshes["Sphere"]
        bpy.data.meshes.remove(mesh)

    # remove mesh Cube
    if "Cube" in bpy.data.meshes:
        mesh = bpy.data.meshes["Cube"]
        bpy.data.meshes.remove(mesh)

    # remove all extra default data

    if "Camera" in bpy.data.cameras:
        camera = bpy.data.cameras["Camera"]
        bpy.data.cameras.remove(camera)


In [ ]:
from happi import Client
from bmad_components.containers import Solenoid, Multipole
import matplotlib.pyplot as plt
from matplotlib import get_backend

db_file = "bmad_db.json"
client = Client(path=db_file)

In [ ]:
# function for generating coords
def calc_coords(range_x, range_y, range_z, padding=0.1):
    focal_length = 15
    angle_of_view_horizontal = 2 * math.atan(36/(2*focal_length))
    angle_of_view_vertical = 2 * math.atan(24/(2*focal_length))
    
    x = np.mean(range_x)
    z = np.mean(range_z)
    
    x_spread = range_x[1] - range_x[0] + padding
    z_spread = range_z[1] - range_z[0] + padding
    
    hypotenuse_horizontal = (x_spread/2)/math.asin(angle_of_view_horizontal/2) 
    hypotenuse_vertical = (z_spread/2)/math.asin(angle_of_view_vertical/2) 
    
    
    if hypotenuse_horizontal > hypotenuse_vertical:
        return x, range_y[0] - hypotenuse_horizontal, z
    
    else:
        return x, range_y[0] - hypotenuse_vertical, z



def render_devices(devices, filename):
    clear_screen()
    
    camera = bpy.data.cameras.new(name='JupyterCamera')
    camera_object = bpy.data.objects.new('JupyterCamera',camera)

    # to place in root container
    bpy.context.scene.collection.objects.link(camera_object)


    range_x = None
    range_y = None
    range_z = None


    objs = []
    for device in devices:

        if device.element_type == "solenoid":
            bpy.ops.mesh.primitive_cube_add()

        elif device.element_type == "multipole":
            bpy.ops.mesh.primitive_uv_sphere_add()

        obj = bpy.context.selected_objects[0]
        if device.L != 0:
            obj.scale = (device.L, device.L, device.L)

        else: 
            #placeholder scale
            obj.scale = (0.1, 0.1, 0.1)

        obj.location = (device.Reference_X, device.Reference_Y, device.Reference_Z)
        objs.append(obj)


        device_x_bounds = [device.Reference_X - obj.scale[0], device.Reference_X + obj.scale[0]]
        device_y_bounds = [device.Reference_Y - obj.scale[1], device.Reference_Y + obj.scale[1]]
        device_z_bounds = [device.Reference_Z - obj.scale[2], device.Reference_Z + obj.scale[2]]

        if not range_x:
            range_x = device_x_bounds

        if not range_y:
            range_y = device_y_bounds

        if not range_z:
            range_z = device_z_bounds


        if device_x_bounds[0] < range_x[0]:
            range_x[0] = device_x_bounds[0]

        if device_x_bounds[1] > range_x[1]:
            range_x[1] = device_x_bounds[1]

        if device_y_bounds[0] < range_y[0]:
            range_y[0] = device_y_bounds[0]

        if device_y_bounds[1] > range_y[1]:
            range_y[1] = device_y_bounds[1]

        if device_z_bounds[0] < range_z[0]:
            range_z[0] = device_z_bounds[0]

        if device_z_bounds[1] > range_z[1]:
            range_z[1] = device_z_bounds[1]
            
    # set up camera
    coords = [range_x[0], range_y[0], range_z[0], range_x[1], range_y[1], range_z[1]]
    camera_object.location = calc_coords(range_x, range_y, range_z)
    camera_object.data.lens = 15
    camera_object.rotation_euler = [math.radians(r) for r in (90,0,0)]
    
    bpy.context.scene.camera = camera_object
    
    bpy.context.scene.render.filepath = filename
    #print(bpy.context.scene.name)
    #bpy.context.view_layer.update()

    bpy.ops.render.opengl(animation=False, render_keyed_only=False, sequencer=False, write_still=True, view_context=False)
    return Image(filename)


In [ ]:
render_devices(client.all_items, "test.png")